In [96]:
import datasets
cor_lines = datasets.readCornellData('', max_len=80, fast_preprocessing=True)

100%|██████████| 83097/83097 [00:05<00:00, 14654.07it/s]


In [3]:
import datasets
cor_lines = datasets.readOpensubsData('open/', max_len= 50, fast_preprocessing=True)

OpenSubtitles data files:   0%|          | 3/2317 [00:00<01:39, 23.24it/s]

Loading OpenSubtitles conversations in open/.
Skipping file open/Horror/1922/1166_134135_184270_nosferatu_eine_symphonie_des_grauens.xml.gz with errors.


OpenSubtitles data files:  15%|█▌        | 352/2317 [00:24<02:15, 14.50it/s]

Skipping file open/Drama/2002/3265_149497_204017_unfaithful.xml.gz with errors.


OpenSubtitles data files:  20%|█▉        | 463/2317 [00:32<02:08, 14.39it/s]

...

OpenSubtitles data files:  83%|████████▎ | 1916/2317 [02:35<00:32, 12.36it/s]

Skipping file open/Comedy/2004/2480_226704_299940_little_black_book.xml.gz with errors.


OpenSubtitles data files:  99%|█████████▉| 2302/2317 [03:14<00:01, 11.83it/s]

Skipping file open/Family/2001/3935_19508_22105_cats__dogs.xml.gz with errors.


100%|██████████| 1648080/1648080 [00:32<00:00, 50508.56it/s]


In [97]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cor_lines, test_size=0.2, random_state=42)

In [5]:
len(cor_lines), train_set[1]

(1115967,
 ('maybe she ran away oh marion',
  'maybe she went into the woods marion i m going 60'))

In [0]:
#from gensim.models.keyedvectors import KeyedVectors                         
#model = KeyedVectors.load_word2vec_format('freebase-vectors-skipgram1000.bin', binary=True)

In [98]:
start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'
unk_symbol = '*'

In [0]:
from collections import Counter
words_counts = Counter('^')
words_counts.update(Counter('$'))
words_counts.update(Counter('#'))
words_counts.update(Counter('*'))
words_counts.update(Counter(' '))
for sequence in train_set:
    words_counts.update(Counter(' '.join(sequence).split()))

words_counts = dict(words_counts)


In [99]:
word2id = {symbol:i for i, symbol in enumerate('^$#*qwertyuioplkjhgfdsamnbvcxz ')}
id2word = {i:symbol for symbol, i in word2id.items()}

In [0]:
word2id = {symbol:i for i, symbol in enumerate(words_counts)}
id2word = {i:symbol for symbol, i in word2id.items()}

In [100]:
start_id = word2id['^']
end_id = word2id['$']
padding_id = word2id['#']
unk_id = word2id['*']
space_id = word2id[' ']

In [101]:
len(word2id)

31

In [102]:
def get_id(x):
    try:
        return word2id[x]
    except:
        return padding_id

In [103]:
def sentence_to_ids(sentence, word2id, padded_len):
    """ Converts a sequence of symbols to a padded sequence of their ids.
    
      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    #sentence = sentence.split()
    l = len(sentence)
    if l  >= padded_len:
        sent_ids = [get_id(symb) for symb in sentence[:padded_len-1]]
        sent_ids.append(1)
        sent_len = padded_len
    else:
        sent_ids = [get_id(symb) for symb in sentence[:padded_len-1]]
        sent_ids.append(1)
        sent_ids.extend([2]*(padded_len - l - 1))
        sent_len = l + 1
    
    return sent_ids, sent_len

In [104]:
sentence_to_ids('have fun tonight goood mooornasd', word2id, 20)

([17, 22, 26, 6, 30, 19, 10, 24, 30, 8, 12, 24, 11, 18, 17, 8, 30, 18, 12, 1],
 20)

In [105]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids] 

In [106]:
ids_to_sentence([17, 22, 26, 6, 30, 19, 10, 24, 30, 8, 12, 24, 11, 18, 17, 8, 30, 18, 12, 1], id2word)

['h',
 'a',
 'v',
 'e',
 ' ',
 'f',
 'u',
 'n',
 ' ',
 't',
 'o',
 'n',
 'i',
 'g',
 'h',
 't',
 ' ',
 'g',
 'o',
 '$']

In [107]:
def batch_to_ids(sentences, word2id, max_len):
    """Prepares batches of indices. 
    
       Sequences are padded to match the longest sequence in the batch,
       if it's longer than max_len, then max_len is used instead.

        sentences: a list of strings, original sequences.
        word2id: a dict, a mapping from original symbols to ids.
        max_len: an integer, max len of sequences allowed.

        result: a list of lists of ids, a list of actual lengths.
    """
    
    max_len_in_batch = min(max(len(s) for s in sentences) + 1, max_len)
    batch_ids, batch_ids_len = [], []
    for sentence in sentences:
        ids, ids_len = sentence_to_ids(sentence, word2id, max_len_in_batch)
        batch_ids.append(ids)
        batch_ids_len.append(ids_len)
    return batch_ids, batch_ids_len

In [108]:
def generate_batches(samples, batch_size=64):
    X, Y = [], []
    for i, (x, y) in enumerate(samples, 1):
        X.append(x)
        Y.append(y)
        if i % batch_size == 0:
            yield X, Y
            X, Y = [], []
    if X and Y:
        yield X, Y

In [109]:
sentences = cor_lines[0]
ids, sent_lens = batch_to_ids(sentences, word2id, max_len =20)
print('Input:', sentences)
print('Ids: {}\nSentences lengths: {}'.format(ids, sent_lens))

Input: ('well i thought wed start with pronunciation if thats okay with you', 'not the hacking and gagging and spitting part please')
Ids: [[5, 6, 14, 14, 30, 11, 30, 8, 17, 12, 10, 18, 17, 8, 30, 5, 6, 20, 30, 1], [24, 12, 8, 30, 8, 17, 6, 30, 17, 22, 27, 15, 11, 24, 18, 30, 22, 24, 20, 1]]
Sentences lengths: [20, 20]


In [110]:
import tensorflow as tf

In [111]:
class Seq2SeqModel(object):
    pass

In [112]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""
    
    # Placeholders for input and its actual lengths.
    self.input_batch = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input_batch')
    self.input_batch_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_batch_lengths')
    
    # Placeholders for groundtruth and its actual lengths.
    self.ground_truth = tf.placeholder(shape = (None, None), dtype = tf.int32, name = 'ground_truth')######### YOUR CODE HERE #############
    self.ground_truth_lengths = tf.placeholder(shape = (None,), dtype = tf.int32, name = 'ground_truth_lengths')######### YOUR CODE HERE #############
        
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    self.learning_rate_ph = tf.placeholder(dtype = tf.float32, shape = [])######### YOUR CODE HERE ############# 

In [113]:
Seq2SeqModel.__declare_placeholders = classmethod(declare_placeholders)

In [114]:
def create_embeddings(self, vocab_size, embeddings_size):
    """Specifies embeddings layer and embeds an input batch."""
     
    random_initializer = tf.random_uniform((vocab_size, embeddings_size), -1.0, 1.0)
    self.embeddings = tf.Variable(initial_value = random_initializer, dtype = tf.float32, name = 'embeddings')######### YOUR CODE HERE ############# 
    
    # Perform embeddings lookup for self.input_batch. 
    self.input_batch_embedded = tf.nn.embedding_lookup(self.embeddings, self.input_batch, name = 'input_batched_embed')######### YOUR CODE HERE ############# 

In [115]:
Seq2SeqModel.__create_embeddings = classmethod(create_embeddings)

In [116]:
def build_encoder(self, hidden_size):
    """Specifies encoder architecture and computes its output."""
    
    # Create GRUCell with dropout.
    #last tf.nn.rnn_cell.GRUCell(num_units = hidden_size, name = 'GRU')
    encoder_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(num_units = hidden_size, name = 'LSTM'), input_keep_prob = self.dropout_ph, dtype = tf.float32)######### YOUR CODE HERE #############
    
    # Create RNN with the predefined cell.
    self.final_encoder_out, self.final_encoder_state = tf.nn.dynamic_rnn(encoder_cell, inputs = self.input_batch_embedded, sequence_length = self.input_batch_lengths, dtype = tf.float32)######### YOUR CODE HERE #############

In [117]:
Seq2SeqModel.__build_encoder = classmethod(build_encoder)

In [121]:
def build_decoder(self, hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id):
    """Specifies decoder architecture and computes the output.
    
        Uses different helpers:
          - for train: feeding ground truth
          - for inference: feeding generated output

        As a result, self.train_outputs and self.infer_outputs are created. 
        Each of them contains two fields:
          rnn_output (predicted logits)
          sample_id (predictions).

    """
    # Use start symbols as the decoder inputs at the first time step.
    batch_size = tf.shape(self.input_batch)[0]
    start_tokens = tf.fill([batch_size], start_symbol_id)
    ground_truth_as_input = tf.concat([tf.expand_dims(start_tokens, 1), self.ground_truth], 1)
    
    # Use the embedding layer defined before to lookup embedings for ground_truth_as_input. 
    self.ground_truth_embedded = tf.nn.embedding_lookup(self.embeddings, ground_truth_as_input, name = 'input_ground_truth_embed')######### YOUR CODE HERE #############
     
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = hidden_size, memory = self.final_encoder_out, memory_sequence_length=self.input_batch_lengths)
    # Create TrainingHelper for the train stage.
    train_helper = tf.contrib.seq2seq.TrainingHelper(self.ground_truth_embedded, 
                                                     self.ground_truth_lengths, name = 'trainer_helper')
    
    # Create GreedyEmbeddingHelper for the inference stage.
    # You should provide the embedding layer, start_tokens and index of the end symbol.
    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(self.embeddings, start_tokens, end_symbol_id)######### YOUR CODE HERE #############
    
     
    def decode(helper, scope, reuse=None):
        """Creates decoder and return the results of the decoding with a given helper."""
        
        with tf.variable_scope(scope, reuse=reuse):
            decoder_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units = hidden_size, reuse = reuse, name = 'lstm_decoder'), input_keep_prob = self.dropout_ph, dtype = tf.float32)######### YOUR CODE HERE #############
            #decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=hidden_size,name = 'AttenWrap')
            # Create a projection wrapper.
            decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=hidden_size,name = 'AttenWrap')
            
            # Create a projection wrapper.
            decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(decoder_cell, vocab_size, reuse = reuse)#reuse
            
            # Create BasicDecoder, pass the defined cell, a helper, and initial state.
            # The initial state should be equal to the final state of the encoder!
            init = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=self.final_encoder_state)
            #init = tf.wrapper.zero_state().clone(cell_state = self.final_encoder_state)
            #decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=hidden_size,name = 'AttenWrap')
            decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state =  init)######### YOUR CODE HERE #############
            
            # The first returning argument of dynamic_decode contains two fields:
            #   rnn_output (predicted logits)
            #   sample_id (predictions)
            outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, maximum_iterations=max_iter, 
                                                              output_time_major=False, impute_finished=True)

            return outputs
        
    self.train_outputs = decode(train_helper, 'decode')
    self.infer_outputs = decode(infer_helper, 'decode', reuse=True)

In [122]:
Seq2SeqModel.__build_decoder = classmethod(build_decoder)

In [126]:
def compute_loss(self):
    """Computes sequence loss (masked cross-entopy loss with logits)."""
    weights = tf.cast(tf.sequence_mask(self.ground_truth_lengths), dtype=tf.float32)
    self.loss = tf.contrib.seq2seq.sequence_loss(logits = self.train_outputs.rnn_output, targets = self.ground_truth, weights = weights)######### YOUR CODE HERE #############

In [127]:
Seq2SeqModel.__compute_loss = classmethod(compute_loss)

In [128]:
def perform_optimization(self):
    """Specifies train_op that optimizes self.loss."""
    params = tf.trainable_variables()
    gradients = tf.gradients(self.loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
    self.train_op = tf.train.AdamOptimizer().apply_gradients(zip(clipped_gradients, params))

In [129]:
Seq2SeqModel.__perform_optimization = classmethod(perform_optimization)

In [130]:
def init_model(self, vocab_size, embeddings_size, hidden_size, 
               max_iter, start_symbol_id, end_symbol_id, padding_symbol_id):
    
    self.__declare_placeholders()
    self.__create_embeddings(vocab_size, embeddings_size)
    self.__build_encoder(hidden_size)
    self.__build_decoder(hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id)
    
    # Compute loss and back-propagate.
    self.__compute_loss()
    self.__perform_optimization()
    
    # Get predictions for evaluation.
    self.train_predictions = self.train_outputs.sample_id
    self.infer_predictions = self.infer_outputs.sample_id

In [131]:
Seq2SeqModel.__init__ = classmethod(init_model)

In [132]:
def train_on_batch(self, session, X, X_seq_len, Y, Y_seq_len, learning_rate, dropout_keep_probability):
    feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: X_seq_len,
            self.ground_truth: Y,
            self.ground_truth_lengths: Y_seq_len,
            self.learning_rate_ph: learning_rate,
            self.dropout_ph: dropout_keep_probability
        }
    pred, loss, _ = session.run([
            self.train_predictions,
            self.loss,
            self.train_op], feed_dict=feed_dict)
    return pred, loss

In [133]:
Seq2SeqModel.train_on_batch = classmethod(train_on_batch)

In [134]:
def predict_for_batch(self, session, X, X_seq_len):
    feed_dict = { self.input_batch: X,
            self.input_batch_lengths: X_seq_len}######### YOUR CODE HERE #############
    pred = session.run([
            self.infer_predictions
        ], feed_dict=feed_dict)[0]
    return pred

def predict_for_batch_with_loss(self, session, X, X_seq_len, Y, Y_seq_len):
    feed_dict = {self.input_batch: X,
            self.input_batch_lengths: X_seq_len,
            self.ground_truth: Y,
            self.ground_truth_lengths: Y_seq_len}######### YOUR CODE HERE #############
    pred, loss = session.run([
            self.infer_predictions,
            self.loss,
        ], feed_dict=feed_dict)
    return pred, loss

In [135]:
Seq2SeqModel.predict_for_batch = classmethod(predict_for_batch)
Seq2SeqModel.predict_for_batch_with_loss = classmethod(predict_for_batch_with_loss)

In [138]:
tf.reset_default_graph()

model = Seq2SeqModel(vocab_size = len(word2id), embeddings_size = 100, max_iter = 50, hidden_size = 500, start_symbol_id = word2id['^'], end_symbol_id = word2id['$'], padding_symbol_id = word2id['#'])######### YOUR CODE HERE #############

batch_size = 60######### YOUR CODE HERE #############
n_epochs = 10######### YOUR CODE HERE #############
learning_rate = 0.001######### YOUR CODE HERE #############
dropout_keep_probability = 0.5######### YOUR CODE HERE #############
max_len = 50######### YOUR CODE HERE #############

n_step = int(len(train_set) / batch_size)

In [139]:
import numpy as np
import random

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())
            
invalid_number_prediction_counts = []
all_model_predictions = []
all_ground_truth = []

print('Start training... \n')
for epoch in range(n_epochs):  
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    print('Train: epoch', epoch + 1)
    for n_iter, (X_batch, Y_batch) in enumerate(generate_batches(train_set, batch_size=batch_size)):
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        # prepare the data (X_batch and Y_batch) for training
        # using function batch_to_ids
        xtr = batch_to_ids(X_batch, word2id, max_len)
        ytr = batch_to_ids(Y_batch, word2id, max_len)
        X_batchh = np.array(xtr[0])
        Y_batchh = np.array(ytr[0])
        X_batch_len = np.array(xtr[1])
        Y_batch_len = np.array(ytr[1])
        predictions, loss = model.train_on_batch(session, X_batchh, X_batch_len, Y_batchh, Y_batch_len, learning_rate, dropout_keep_probability )
        
        #model.predict_for_batch_with_loss(session=session,X =  X_batchh, X_seq_len=X_batch_len, Y=Y_batchh, Y_seq_len=Y_batch_len)######### YOUR CODE HERE #############
        
        if n_iter % 200 == 0:
            print("Epoch: [%d/%d], step: [%d/%d], loss: %f" % (epoch + 1, n_epochs, n_iter + 1, n_step, loss))
                
    X_sent, Y_sent = next(generate_batches(test_set, batch_size=batch_size))
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    # prepare test data (X_sent and Y_sent) for predicting 
    # quality and computing value of the loss function
    # using function batch_to_ids
    xs = batch_to_ids(X_sent, word2id, max_len)
    ys = batch_to_ids(Y_sent, word2id, max_len)
    X_batchhs = np.array(xs[0])
    Y_batchhs = np.array(ys[0])
    X_batch_lens = np.array(xs[1])
    Y_batch_lens = np.array(ys[1])
    predictions, loss = model.predict_for_batch_with_loss(session, X_batchhs, X_batch_lens, Y_batchhs, Y_batch_lens)######### YOUR CODE HERE #############
    print('Test: epoch', epoch + 1, 'loss:', loss,)
    for x, y, p  in list(zip(X_batchhs, Y_batchhs, predictions))[:10]:
        print('X:',''.join(ids_to_sentence(x, id2word)))
        print('Y:',''.join(ids_to_sentence(y, id2word)))
        print('O:',''.join(ids_to_sentence(p, id2word)))
        print('')

    # ground-truth value to the arrays.
print('\n...training finished.')

Start training... 

Train: epoch 1
Epoch: [1/10], step: [1/1998], loss: 3.440839
Epoch: [1/10], step: [201/1998], loss: 1.720613
Epoch: [1/10], step: [401/1998], loss: 1.543447
Epoch: [1/10], step: [601/1998], loss: 1.513597
Epoch: [1/10], step: [801/1998], loss: 1.398102
Epoch: [1/10], step: [1001/1998], loss: 1.413745
Epoch: [1/10], step: [1201/1998], loss: 1.330769
Epoch: [1/10], step: [1401/1998], loss: 1.416018
Epoch: [1/10], step: [1601/1998], loss: 1.337224
Epoch: [1/10], step: [1801/1998], loss: 1.382533
Test: epoch 1 loss: 1.3547747
X: frank uh you know ill try to contribute but uhfre$
Y: i dont care what it costs this is when ya come to$
O: i dont know what i said i dont know what i said$^^

X: its saturday arnie you work too hard$#############
Y: carl im running late im coming right now$#########
O: i dont know what i said i dont know what i said$^^

X: its battle i want not talk$#######################
Y: but now that i am here will you speak with a woma$
O: what do you wan

Epoch: [5/10], step: [1/1998], loss: 1.168151
Epoch: [5/10], step: [201/1998], loss: 1.152520
Epoch: [5/10], step: [401/1998], loss: 1.228279
Epoch: [5/10], step: [601/1998], loss: 1.240051
Epoch: [5/10], step: [801/1998], loss: 1.187684
Epoch: [5/10], step: [1001/1998], loss: 1.215816
Epoch: [5/10], step: [1201/1998], loss: 1.210794
Epoch: [5/10], step: [1401/1998], loss: 1.230243
Epoch: [5/10], step: [1601/1998], loss: 1.224180
Epoch: [5/10], step: [1801/1998], loss: 1.204566
Test: epoch 5 loss: 1.1848286
X: kristen what happened$############################
Y: youll hear all kinds of stories theyll tell you i$
O: i dont know i know i know i know i know i know i$^

X: i need an exit fast$##############################
Y: cypher$###########################################
O: i dont know what you want to do with me$^^^^^^^^^^

X: but i just got here$##############################
Y: youve been here longer than you think$############
O: i dont know what you just want to go to the couc$^

In [217]:
seq = ['what is your name']
seq_ids = batch_to_ids(seq, word2id, max_len = 5)
predictions= model.predict_for_batch(session, np.array(seq_ids[0][0]).reshape(1, -1), np.array(seq_ids[1]))
#print('X:',''.join(ids_to_sentence(np.array(seq_ids[0][0]).reshape(1, -1), id2word)))
print('O:',''.join(ids_to_sentence(predictions[0], id2word)))
print('')

O: idontknow$

